# The Battle of Neighborhoods

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors



Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

## Getting the latitude and Longitude for Delhi

In [3]:
address = 'Delhi, IN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi are 28.6517178, 77.2219388.


Connecting to foursquare

In [4]:
CLIENT_ID = 'TPPFPP3AMBVM2JBUXAYLVU15AREYMPRXHI3NNGYLFUCYFLAB' # your Foursquare ID
CLIENT_SECRET = 'TDYXKE3LE2AAM3GV42ZHS5VKDZFVPV32SNLYMZ3CW10HBSAL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: TPPFPP3AMBVM2JBUXAYLVU15AREYMPRXHI3NNGYLFUCYFLAB
CLIENT_SECRET:TDYXKE3LE2AAM3GV42ZHS5VKDZFVPV32SNLYMZ3CW10HBSAL


Using the latitude and longitude of Hudson Lane to get the Cafes within the radius of 2.5 KM

In [5]:


LIMIT = 100
latitude=28.6946
longitude=77.2053
radius = 2500
query='cafe'
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude,
    query,
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=TPPFPP3AMBVM2JBUXAYLVU15AREYMPRXHI3NNGYLFUCYFLAB&client_secret=TDYXKE3LE2AAM3GV42ZHS5VKDZFVPV32SNLYMZ3CW10HBSAL&v=20180605&ll=28.6946,77.2053&query=cafe&radius=2500&limit=100'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ce6c8746a6071493bfd9363'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-513a1684e4b0581da224735e-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16d941735',
         'name': 'Café',
         'pluralName': 'Cafés',
         'primary': True,
         'shortName': 'Café'}],
       'id': '513a1684e4b0581da224735e',
       'location': {'address': 'Hudson Lane',
        'cc': 'IN',
        'city': 'Delhi',
        'country': 'India',
        'crossStreet': 'opposite NDPL',
        'distance': 154,
        'formattedAddress': ['Hudson Lane (opposite NDPL)',
         'Delhi 110009',
         'Delhi',
         'India'],
        'labeledLatLngs': [{'label': 'display',


In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    


In [8]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.formattedAddress']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng,formattedAddress
0,Big Yellow Door,Café,28.693245,77.204948,"[Hudson Lane (opposite NDPL), Delhi 110009, De..."
1,Woodbox Cafe,Café,28.694334,77.204811,"[Hudson Lane, 1 DDA Market,behind NDPL office,..."
2,Ama café,Café,28.702568,77.228266,"[Majnu Ka Tilla, India]"
3,Dunkin Donuts,Café,28.705037,77.190069,[2 Block B Model Town 2 Central Market (Ring R...
4,Shake Square,Café,28.681606,77.207240,"[Bungalow road (Kamla Nagar), New Delhi 110007..."
5,The Vintage Avenue,Café,28.694723,77.204410,"[Hudson Lane (Next to PP Jewellers), Delhi, De..."
6,Central Perk,Café,28.694052,77.202001,"[2510 hudson lane (Gtb nagar), New Delhi 11000..."
7,AIM cafe,Café,28.701137,77.209033,[India]
8,CCD kamla nagar,Café,28.681796,77.207049,"[Kamla Nagar, India]"
9,Cafe Coffee Day,Café,28.696103,77.179944,"[Ashok Vihar, India]"


Visualizing the top Cafes Using a Map

In [10]:

map_HudsonLane = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng in zip(nearby_venues['lat'], nearby_venues['lng']):
    #label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
       # popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_HudsonLane)  
    
map_HudsonLane

2nd PLace-Hauz Khas

Using the latitude and longitude of Hauz Khas to get the Cafes within the radius of 2.5 KM

In [11]:
# type your answer here

LIMIT = 100
latitude_2=28.5479
longitude_2=77.2031
query = 'cafe'
radius=3000
url_2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_2, 
    longitude_2,
    query,
    radius, 
    LIMIT)
url_2


'https://api.foursquare.com/v2/venues/explore?&client_id=TPPFPP3AMBVM2JBUXAYLVU15AREYMPRXHI3NNGYLFUCYFLAB&client_secret=TDYXKE3LE2AAM3GV42ZHS5VKDZFVPV32SNLYMZ3CW10HBSAL&v=20180605&ll=28.5479,77.2031&query=cafe&radius=3000&limit=100'

In [12]:
results_2 = requests.get(url_2).json()
results_2

{'meta': {'code': 200, 'requestId': '5ce6c892351e3d12861f4f11'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5332a162498edc37012c3268-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16d941735',
         'name': 'Café',
         'pluralName': 'Cafés',
         'primary': True,
         'shortName': 'Café'}],
       'id': '5332a162498edc37012c3268',
       'location': {'address': '1st Floor, Aurobindo Place Market',
        'cc': 'IN',
        'city': 'New Delhi',
        'country': 'India',
        'distance': 505,
        'formattedAddress': ['1st Floor, Aurobindo Place Market',
         'New Delhi 110016',
         'Delhi',
         'India'],
        'labeledLatLngs': [{'label': 'display',
    

In [13]:
venues_2 = results_2['response']['groups'][0]['items']
    
nearby_venues_2 = json_normalize(venues_2) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.formattedAddress']
nearby_venues_2 =nearby_venues_2.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_2['venue.categories'] = nearby_venues_2.apply(get_category_type, axis=1)

# clean columns
nearby_venues_2.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues_2

,name,categories,lat,lng,formattedAddress
0,Summer House Cafe,Café,28.552401,77.203766,"[1st Floor, Aurobindo Place Market, New Delhi ..."
1,Coast Cafe,Café,28.554779,77.195214,"[Hauz Khas Village, New Delhi, Delhi, India]"
2,Diggin,Café,28.555665,77.218590,[1&2 Anand Lok shopping center (Opp Gargi coll...
3,Social Offline,Café,28.554461,77.193881,"[9A & 12, Hauz Khas Village, New Delhi 110016,..."
4,Raasta,Café,28.553829,77.194764,"[30, Hauz Khas Village (Above Power House Gym)..."
5,Kettle Drum,Café,28.546677,77.196841,[India]
6,The Potbelly Rooftop Cafe & Kitchen,Café,28.549666,77.215340,"[116 C, 4th Flr., Shahpur Jat (Near Slice of I..."
7,The Wishing Chair / The Mad Teapot Cafe,Café,28.549288,77.214362,"[86 A, Shahpur Jat, Delhi, Delhi, India]"
8,Adyar Ananda Bhavan,Café,28.557293,77.202502,"[Green Park Market, New Delhi 110016, Delhi, I..."
9,T'POT,Café,28.537433,77.210413,"[Plot no 7, Opposite Kotak Mahindra Bank, Malv..."


Visualizing the top Cafes Using a Map

In [14]:
map_HauzKhas = folium.Map(location=[latitude_2, longitude_2], zoom_start=11)

# add markers to map
for lat, lng in zip(nearby_venues_2['lat'], nearby_venues_2['lng']):
    #label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
       # popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_HauzKhas)  
    
map_HauzKhas